# Day 9: 지역 분류 및 클러스터링

**날짜**: 2025-07-12

**목표**:
- 4분면 분류 (CCTV 밀도 vs 범죄율)
- 우선순위 지역 식별
- 정책 타겟 그룹 도출

In [ ]:
import sys
import os
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from utils import *

set_korean_font()
set_plot_style()

df = pd.read_csv(os.path.join(DATA_PATHS['processed'], 'integrated_data.csv'), encoding='utf-8-sig')
print(f"✅ Day 9: 지역 분류 및 클러스터링")

## 1. 4분면 분류 (CCTV 밀도 vs 범죄율)

In [ ]:
# 중앙값 기준으로 4분면 분류
cctv_median = df['인구당_방범용'].median()
crime_median = df['인구당_CCTV효과범죄율'].median()

def classify_quadrant(row):
    cctv = row['인구당_방범용']
    crime = row['인구당_CCTV효과범죄율']
    
    if cctv >= cctv_median and crime >= crime_median:
        return 'Q1: 고CCTV/고범죄'
    elif cctv < cctv_median and crime >= crime_median:
        return 'Q2: 저CCTV/고범죄 (우선순위)'
    elif cctv < cctv_median and crime < crime_median:
        return 'Q3: 저CCTV/저범죄'
    else:
        return 'Q4: 고CCTV/저범죄 (효과적)'

df['분면'] = df.apply(classify_quadrant, axis=1)

print("4분면 분류 결과:")
print(df['분면'].value_counts())
print(f"\n중앙값:")
print(f"  방범용 CCTV: {cctv_median:.2f}대/천명")
print(f"  범죄율: {crime_median:.2f}건/천명")

## 2. 4분면 시각화

In [ ]:
# 분면별 색상 매핑
color_map = {
    'Q1: 고CCTV/고범죄': 'orange',
    'Q2: 저CCTV/고범죄 (우선순위)': 'red',
    'Q3: 저CCTV/저범죄': 'lightblue',
    'Q4: 고CCTV/저범죄 (효과적)': 'green'
}

fig, ax = plt.subplots(figsize=(14, 10))

for quadrant, color in color_map.items():
    subset = df[df['분면'] == quadrant]
    ax.scatter(subset['인구당_방범용'], subset['인구당_CCTV효과범죄율'],
               s=200, alpha=0.7, edgecolors='black', linewidth=1.5,
               color=color, label=quadrant)
    
    # 자치구 이름 라벨
    for idx, row in subset.iterrows():
        ax.annotate(row['자치구'], 
                   (row['인구당_방범용'], row['인구당_CCTV효과범죄율']),
                   fontsize=9, ha='center', va='bottom')

# 중앙값 기준선
ax.axvline(cctv_median, color='gray', linestyle='--', linewidth=2, alpha=0.5)
ax.axhline(crime_median, color='gray', linestyle='--', linewidth=2, alpha=0.5)

ax.set_xlabel('인구당 방범용 CCTV (대/천명)', fontsize=12, fontweight='bold')
ax.set_ylabel('인구당 CCTV효과범죄율 (건/천명)', fontsize=12, fontweight='bold')
ax.set_title('자치구 4분면 분류: CCTV 밀도 vs 범죄율', fontsize=14, fontweight='bold')
ax.legend(loc='best', fontsize=10)
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(DATA_PATHS['figures'], 'day9_quadrant_classification.png'), dpi=300)
plt.show()

## 3. 우선순위 지역 분석

In [ ]:
# Q2: 저CCTV/고범죄 (우선순위 설치 필요)
priority_districts = df[df['분면'] == 'Q2: 저CCTV/고범죄 (우선순위)'].sort_values('인구당_CCTV효과범죄율', ascending=False)

print("="*70)
print("우선순위 설치 지역 (Q2: 저CCTV/고범죄)")
print("="*70)
if len(priority_districts) > 0:
    print(priority_districts[['자치구', '인구당_방범용', '인구당_CCTV효과범죄율', '인구밀도']].to_string(index=False))
    print(f"\n총 {len(priority_districts)}개 자치구")
else:
    print("해당 없음")

# Q4: 고CCTV/저범죄 (효과적인 사례)
effective_districts = df[df['분면'] == 'Q4: 고CCTV/저범죄 (효과적)'].sort_values('인구당_방범용', ascending=False)

print("\n" + "="*70)
print("CCTV 효과적인 지역 (Q4: 고CCTV/저범죄)")
print("="*70)
if len(effective_districts) > 0:
    print(effective_districts[['자치구', '인구당_방범용', '인구당_CCTV효과범죄율', '인구밀도']].to_string(index=False))
    print(f"\n총 {len(effective_districts)}개 자치구")
else:
    print("해당 없음")

## 4. 분면별 통계 요약

In [ ]:
# 분면별 평균값
quadrant_stats = df.groupby('분면').agg({
    '인구당_방범용': 'mean',
    '인구당_CCTV효과범죄율': 'mean',
    '인구밀도': 'mean',
    '자치구': 'count'
}).round(2)

quadrant_stats.columns = ['평균_방범용CCTV', '평균_범죄율', '평균_인구밀도', '자치구수']

print("\n분면별 통계 요약:")
print(quadrant_stats)

# 결과 저장
df.to_csv(os.path.join(DATA_PATHS['processed'], 'integrated_data_with_quadrant.csv'), 
          index=False, encoding='utf-8-sig')

quadrant_stats.to_csv(os.path.join(DATA_PATHS['reports'], 'day9_quadrant_stats.csv'), 
                     encoding='utf-8-sig')

## 5. 정책 시사점

In [ ]:
insights = f"""
📊 지역 분류 기반 정책 시사점

1. Q2 (저CCTV/고범죄): 우선순위 설치 지역
   - 자치구 수: {len(priority_districts)}개
   - 특징: 범죄율 높지만 CCTV 밀도 낮음
   - 정책: 방범용 CCTV 집중 설치 필요

2. Q4 (고CCTV/저범죄): 효과적인 사례
   - 자치구 수: {len(effective_districts)}개
   - 특징: CCTV 밀도 높고 범죄율 낮음
   - 정책: 모범 사례로 벤치마킹 대상

3. Q1 (고CCTV/고범죄): 추가 대책 필요
   - 특징: CCTV만으로 범죄 억제 한계
   - 정책: 조명, 순찰 등 종합적 방범 대책 필요

4. Q3 (저CCTV/저범죄): 현상 유지
   - 특징: 범죄율 낮아 현재 수준 유지
   - 정책: 급격한 증설 불필요, 모니터링 중심
"""

print(insights)

# 파일 저장
with open(os.path.join(DATA_PATHS['reports'], 'day9_policy_insights.txt'), 'w', encoding='utf-8') as f:
    f.write(insights)

In [ ]:
print("✅ Day 9 완료")